In [1]:
import numpy as np
import pandas as pd
import random
import string
import timeit

In [191]:
class Agent():
    '''
    This class embodies a single individual, "Agent", within the greater system.
    This Agent has characteristics that help to determine their probability for actions.
    '''

    def __init__(self):
        self.see_probability = None
        self.share_probability = None
        self.agent_id = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
        self.connections = []

    def update_probabilities(self, characteristic, value):
        '''A function used to update the characteristics of this particular agent
        Args:
            characteristic: the name of the characteristic to be updated, str
            value: the value to update 'characteristic' to
        Returns:
            an updated characteristic of the Agent
        '''
        if characteristic == 'see_probability':
            self.see_probability = value

        if characteristic == 'share_probability':
            self.share_probability = value

    def add_connections(self, connections_list):
        '''A function to add connections to the Agent
        Args:
            connections_list: a list of other agent_id's
        Retuns:
            an updated self.connections
        '''
        # Check to ensure that connections_list is, indeed, a list. Otherwise fuckery
        if type(connections_list) != list:
            connections_list = [connections_list]
        
        # Looping through the list and adding other Agent's agent_ids
        for con in connections_list:
            # We need to ensure that they are not already in the connections list
            # In addition, we need to ensure that and Agent cannot be connected to theirself
            if con in self.connections:
                continue
            else:
                if con == self.agent_id:
                    continue
                else:
                    self.connections.append(con)        
        
        
class Network():
    '''
    This class holds the information about the directional connection of the network itself.
    '''

    def __init__(self):
        self.agents = None

    def add_agents(self, agents):
        '''A helper function to add a list of agents to the network
        Args:
            the agent_ids of the agents, list
        '''
        self.agents = agents
        
    def find_agent(self, agent_id):
        '''A helper function to find a specific agent in the Network
        Args:
            agent_id: the ID of the agent you are trying to find, str
        Returns:
            an agent, Agent()
        '''
        # Handling case where agent_id is not in the network
        # Using list comprehension for speed, grab the Agent that matches agent_id
        results = [agent for agent in agents if agent.agent_id == agent_id]
        
        # If the len of the returned list is 0, Agent not found in this network
        # Else, return the Agent
        if len(results) == 0:
            raise AttributeError(f'{agent_id} is not in this network.')
        else:
            return results[0]
    
    def create_connections(self, connections):
        '''A helper function that adds connections to agents
        Args:
            connections: list of tuples of connections, [(agent1, agent2), (agent2, agent3)]
        '''
        # Grabbing a list of our agent_ids for reference
        network_agents = [agent.agent_id for agent in self.agents]

        # Looping through the outer list of tuples of the connections
        for con in connections:
            
            # We need to ensure that each agent_id exists for an Agent in this network
            if any([con[0] not in network_agents, con[1] not in network_agents]):
                raise AttributeError(f'Either {con[0]} or {con[1]} is not in this network.')
                continue
            # Grabbing the first agent in the tuple to add the second
            # Then vice-versa
            self.find_agent(agent_id=con[0]).add_connections(connections_list=con[1])
            self.find_agent(agent_id=con[1]).add_connections(connections_list=con[0])
        
    def simulate_seen(self, initalizer, n_simulations):
        '''A function that simulates the spread of information through this network
        Args:
            initalizer: the agent_id of the Agent that makes the initial post, str
            n_simulations: the number of simulations desired, int
        '''
        # Asserts
        assert type(n_simulations) == int, 'n_sumulations must be an integer'
        
        # First need to ensure that the 'initalizer' is in this Network
        # find_agent() raises error if agent_id not in network, so we rely on that function
        initalizer_agent = self.find_agent(agent_id=initalizer)
        
        # We can save some compute by grabbing a list of Agents from the initalizer up front
        initalizer_connections = [self.find_agent(agent_id=con) for con in initalizer_agent.connections]
        
        # Outer loop of the number of simulations
        for n in range(n_simulations):
            
            # First, we loop thru the connections of the initalizer_agent and calculate
            # the probability of each Agent seeing the post
            # Then, we calculate the probability of them sharing the post
            first_level_seen = [probability(n=1, probability_failure=(100 - agent.see_probability))[0] for agent in initalizer_connections]
            first_level_share = [probability(n=1, probability_failure=(100 - agent.share_probability))[0] for agent in initalizer_connections]
            
            # Now we can calculate the probability of these agents sharing GIVEN
            # that they have seen the original post
            first_level_pass = [1 if i==1 and j==1 else 0 for i, j in zip(first_level_seen,
                                                                         first_level_share)]
            return first_level_pass

    def simulate_seen2(self, initalizer):
        '''A function that simulates the spread of information through this network
        Args:
            initalizer: the agent_id of the Agent that makes the initial post, str
            n_simulations: the number of simulations desired, int
        '''
        # Asserts
        assert type(n_simulations) == int, 'n_sumulations must be an integer'
        
        # First need to ensure that the 'initalizer' is in this Network
        # find_agent() raises error if agent_id not in network, so we rely on that function
        initalizer_agent = self.find_agent(agent_id=initalizer)
        
        # We can save some compute by grabbing a list of Agents from the initalizer up front
        initalizer_connections = [self.find_agent(agent_id=con) for con in initalizer_agent.connections]
        
        #
        
    
                                  
def probability(n, probability_failure):
    return [1 if num > probability_failure else 0 for num in [random.randint(0, 100) for x in range(n)]]

In [175]:
a = [1, 1, 0, 0]
b = [1, 0, 1, 0]

[1 if i==1 and j==1 else 0 for i, j in zip(a, b)]

[1, 0, 0, 0]

In [231]:
agents = [Agent() for i in range(15)]
network = Network()

for agent in agents:
    agent.update_probabilities(characteristic='see_probability', value=random.randint(0, 100))
    
for agent in agents:
    agent.update_probabilities(characteristic='share_probability', value=random.randint(0, 100))
    
network.add_agents(agents=agents)

In [232]:
for agent in network.agents:
    print(agent.agent_id, agent.see_probability, agent.share_probability)

8JeE6IS8XdJDXOma 17 88
b0DBZOJoAnC4ERCL 70 57
gUTA6eu1FMTnvcZY 34 45
Hw7KbXMM6Cuba4Ox 10 26
UhTsEjvsKSym6niy 38 96
5iH1j5PqAbVUb4HJ 60 65
e1gKuv03oA59D4jS 23 20
DpO2VyU7o4p9oOaM 46 65
QmRSdo2kcnAUGfTG 55 38
L6Yog1s2puA2qWnE 88 83
YP11SEBG93s2bJok 10 37
Giv67wvCVoZBCTAw 73 98
6AlCO1nUVvITq1Uu 24 4
6dpi0HuN2NZXcmVk 57 50
PDFx8FIsFSk73vTH 63 69


In [234]:
network.create_connections(connections=[
    ('8JeE6IS8XdJDXOma', 'b0DBZOJoAnC4ERCL'),
    ('8JeE6IS8XdJDXOma', 'gUTA6eu1FMTnvcZY'),
    ('8JeE6IS8XdJDXOma', 'Hw7KbXMM6Cuba4Ox'),
    ('8JeE6IS8XdJDXOma', 'UhTsEjvsKSym6niy'),
    ('8JeE6IS8XdJDXOma', '5iH1j5PqAbVUb4HJ'),
])

In [235]:
network.create_connections(connections=[
    ('b0DBZOJoAnC4ERCL', 'e1gKuv03oA59D4jS'),
    ('b0DBZOJoAnC4ERCL', 'DpO2VyU7o4p9oOaM'),
])

network.create_connections(connections=[
    ('UhTsEjvsKSym6niy', 'QmRSdo2kcnAUGfTG')
])

network.create_connections(connections=[
    ('gUTA6eu1FMTnvcZY', 'L6Yog1s2puA2qWnE'),
    ('gUTA6eu1FMTnvcZY', 'YP11SEBG93s2bJok')
])

network.create_connections(connections=[
    ('Hw7KbXMM6Cuba4Ox', '6dpi0HuN2NZXcmVk'),
    ('Hw7KbXMM6Cuba4Ox', '6AlCO1nUVvITq1Uu'),
    ('Hw7KbXMM6Cuba4Ox', 'Giv67wvCVoZBCTAw')
])

network.create_connections(connections=[
    ('5iH1j5PqAbVUb4HJ', 'PDFx8FIsFSk73vTH')
])

In [ ]:
def depth_simulation(initalizer):
    '''A functions that simulates a Network our to a given layer depth
    Args:
        initalizer: the agent_id of the Agent that makes the initial post, str
    '''

In [1]:
def layered_connections(network, initalizer, n_layers):
    '''A helper function that finds all of the connections out to n_layers layers
    Args:
        initalizer: the agent_id of the Agent that begins the search, str
        n_layers: the number of layers to search through
    Returns:
        asdasd
    '''
    # Asserting that the given initalizer Agent exists in the network
    initalizer_agent = network.find_agent(agent_id=initalizer)
  
    # First, create our dictionary with our first layer connections (from initalizer Agent)
    connections_dict = {
        0: {initalizer: [network.find_agent(agent_id=agent) for agent in initalizer_agent.connections]}
    }

    # Looping through the number of layers in n_layers
    # Start loop at one, as we reference [i-1] to get the Agents to search through for i
    for i in range(1, n_layers):

        # First, grab a list of the Keys (agent_ids, str) from the previous layer
        prev_layer_agent_ids = list(connections_dict[i-1].keys())

        # Then, we need to initalize another Key for this level
        connections_dict[i] = {}
        
        # Then, iterate through this list to grab the relevant Agents 
        # Once we have those Agents, grab all of their .connections
        # Finally, append this information to our dictionary
        for agent_id in prev_layer_agent_ids:
            
            # Grabbing and holding our Agent objects in memory
            Agent_holder = connections_dict[i-1][agent_id]
            
            # Looping through each Agent in our Agent_holder
            # and adding the Agent's agent_id as a key and their connections 
            # that are NOT 
            for Agent in Agent_holder:
                connections_dict[i][Agent.agent_id] = [network.find_agent(agent_id=ai) for ai in Agent.connections if ai not in prev_layer_agent_ids]
    
    return connections_dict               

In [347]:
con_dict = layered_connections(initalizer='8JeE6IS8XdJDXOma', n_layers=3)
con_dict

1 

2 



{0: {'8JeE6IS8XdJDXOma': [<__main__.Agent at 0x159b9833c50>,
   <__main__.Agent at 0x159b9833630>]},
 1: {'b0DBZOJoAnC4ERCL': [<__main__.Agent at 0x159b9833048>,
  'gUTA6eu1FMTnvcZY': [<__main__.Agent at 0x159b98330b8>,
  'Hw7KbXMM6Cuba4Ox': [<__main__.Agent at 0x159b9833908>,
  'UhTsEjvsKSym6niy': [<__main__.Agent at 0x159b9833438>],
  '5iH1j5PqAbVUb4HJ': [<__main__.Agent at 0x159b9833eb8>]},
 2: {'e1gKuv03oA59D4jS': [],
  'DpO2VyU7o4p9oOaM': [],
  'L6Yog1s2puA2qWnE': [],
  'YP11SEBG93s2bJok': [],
  '6dpi0HuN2NZXcmVk': [],
  '6AlCO1nUVvITq1Uu': [],
  'Giv67wvCVoZBCTAw': [],
  'QmRSdo2kcnAUGfTG': [],
  'PDFx8FIsFSk73vTH': []}}

In [2]:
con_dict = layered_connections(network=network, initalizer='8JeE6IS8XdJDXOma', n_layers=3)
con_dict

NameError: name 'network' is not defined

In [306]:
a = {'this': {'inside': [1, 2, 3]}}
print(a)

a['that'] = {'inside2': [4, 5, 6]}
print(a)

{'this': {'inside': [1, 2, 3]}}
{'this': {'inside': [1, 2, 3]}, 'that': {'inside2': [4, 5, 6]}}


In [345]:
a = {1: {'this': [1, 2, 3]}}
print(a)

a[2] = {}
print(a)

a[2]['that'] = [4, 5, 6]
print(a)

{1: {'this': [1, 2, 3]}}
{1: {'this': [1, 2, 3]}, 2: {}}
{1: {'this': [1, 2, 3]}, 2: {'that': [4, 5, 6]}}


In [250]:
dict(['this', 'that'])

ValueError: dictionary update sequence element #0 has length 4; 2 is required

In [237]:
network.find_agent(agent_id='8JeE6IS8XdJDXOm')

AttributeError: 8JeE6IS8XdJDXOm is not in this network.

In [224]:
print(.85*.75)
print(.06*.19)
print(.11*.24)

0.6375
0.0114
0.0264


In [230]:
l = []

for n in range(10000):
    l.append(network.simulate_seen(initalizer='zvULY9UBVKaAiyI5', n_simulations=1))

# 

In [227]:
l2 = []
for i in range(3):
    l2 = []
#     print('HERE')
    for n in l:
        l2.append(n[i])
        
    print(np.mean(l2))

0.62438
0.010927
0.026004


In [207]:
network.find_agent(agent_id='zvULY9UBVKaAiyI5').connections

['ItOAGB028mvgsZPe', 'tpUvbh0XIVwvzKaa', 'gSQcZeEmUwQ9OBXJ']

In [206]:
network.create_connections(connections=[('zvULY9UBVKaAiyI5', 'ItOAGB028mvgsZPe'),
                                       ('zvULY9UBVKaAiyI5', 'tpUvbh0XIVwvzKaa'),
                                       ('zvULY9UBVKaAiyI5', 'gSQcZeEmUwQ9OBXJ')])

In [183]:
network.simulate_seen(initalizer='zvULY9UBVKaAiyI5', n_simulations=1)

Seen: [1, 1] 
 Shared: [0, 1] 
 Pass: [0, 1]


In [138]:
for agent in network.agents:
    print(agent.see_probability)

88
19
38
14
61
11
100
14
53
63
23
28
81


In [134]:
[probability(n=1, probability_failure=(100 - agent.see_probability))[0] for agent in network.agents]

[0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]

In [127]:
network.simulate_seen(initalizer='Eg0nLfmnh8uiHiBz')

In [98]:
network.create_connections(connections=[('5T76t4jW6cX0x2Rw', 'OWBKiMnerONPfdaG'),
                                       ('OWBKiMnerONPfdaG', '6HyB9I4extP5bVc3')])

In [99]:
print(network.find_agent(agent_id='5T76t4jW6cX0x2Rw').connections)
print(network.find_agent(agent_id='OWBKiMnerONPfdaG').connections)
print(network.find_agent(agent_id='6HyB9I4extP5bVc3').connections)

['OWBKiMnerONPfdaG']
['5T76t4jW6cX0x2Rw', '6HyB9I4extP5bVc3']
['OWBKiMnerONPfdaG']


In [71]:
a = [1, 2, 3, 4]
if any([2 not in a, 3 not in a]):
    print('herer')

In [106]:
network.find_agent(agent_id='WEov4TQfsBwltqH3')

In [ ]:
network.find_agent(agent_id='XC4ZXS5JfqkJabdz').add_connections(connections_list=[
    'JuqWGq5bA18NwLec',
    'C5SzWXXqztjAhjRE',
    'fDPk8Trw7PbGE2e2'
])